## 캐릭터 이미지 씌우기

## 씨발 완성

In [5]:
import cv2
import mediapipe as mp

#동영상이 3채널이고 이미지가 4채널이라서 적용이 안됨. 이를 해결해주기 위해 쓴 함수
#겹치는 함수
def overlay(image, x, y, w, h, overlay_image): #대상이미지(3채널),x,y좌표, width, height, 덮어씌울 이미지(4채널)
    alpha=overlay_image[:, :, 3] #BGRA
    mask_image=alpha/255 #0~255로 나누면 0~1사이의 값(1은 불투명, 0은 완전 투명)

    for c in range(0,3): #channel BGR
        image[y-h:y+h, x-w:x+w, c]= (overlay_image[:,:,c]*mask_image) + (image[y-h:y+h, x-w:x+w, c]*(1-mask_image))
        #1-mask_image하면 반전이 됨. 

        
# MediaPipe 얼굴 인식 초기화
mp_face_detection = mp.solutions.face_detection #얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils #얼굴의 특징을 그리기 위한 drawing_utils모듈을 사용

# 비디오 캡처(동영상 파일 열기)
cap=cv2.VideoCapture('face_video.mp4')

# 이미지 불러오기
image_right_eye = cv2.imread('right_eye_trans.png', cv2.IMREAD_UNCHANGED)#오른쪽 귀 100*100
image_left_eye = cv2.imread('left_eye_trans.png', cv2.IMREAD_UNCHANGED) #왼쪽 귀 100*100
image_nose = cv2.imread('nose_trans.png', cv2.IMREAD_UNCHANGED) #코(고양이수염) 300*100(가로*세로)

# 비디오 프레임 처리
#model_selection=0or1(0은 2m이내의 근거리, 1은 5m이내의 원거리 )
#min_detection_confidence=0.5는 50%이상의 신뢰도로 여기에 얼굴이 있는 것 같아. 라고 생각함.(임계치 개념)
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    #제대로 동영상이 열렸는가?
    while cap. isOpened():
        success, image=cap.read()
        if not success:
            break
            
        image.flags.writeable=False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#cv2.cvtColor: 프레임의 색상을 BGR에서 RGB로 변환
        results=face_detection.process(image) #변환된 이미지에서 얼굴을 인식
        
        image.flags.writeable=True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        #검출된 얼굴이 있다면? 그림 그리기
        if results.detections:
            #6개 특징: 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀(귀 구슬점, 이주(귓구멍 앞부분)) 추출
            for detection in results.detections:
                #흰색 네모박스+6개의 포인트에 빨간 점 찍기
                #mp_drawing.draw_detection(image, detection) #인식된 얼굴에 대해 시각적인 표시
                print(detection) #추출된 특징을 print로 확인하기
                
                #특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye=keypoints[0] #오른쪽 눈
                left_eye=keypoints[1] #왼쪽 눈
                nose_tip=keypoints[2] #코 끝부분 
                
                #이미지의 크기 정보를 가지고 좌표 계산하기
                h,w,_=image.shape #height, width, channel: 이미지로부터 세로, 가로 크기 가져옴
                right_eye=(int(right_eye.x*w)-20, int(right_eye.y*h)-100) #좌표값을 조금 변경해서 귀가 눈보다 위에있게하기
                left_eye=(int(left_eye.x*w)+20, int(left_eye.y*h)-100) #
                nose_tip=(int(nose_tip.x*w), int(nose_tip.y*h))
                
                #양 눈에 동그라미 그리기 
                #cv2.circle(image, right_eye, 50, (255,0,0), 10, cv2.LINE_AA) #파란색 동그라미
                #cv2.circle(image, left_eye, 50, (0,255,0), 10, cv2.LINE_AA)
                
                #코에 동그라미 그리기
                #cv2.circle(image, nose_tip, 30, (0,255,255), 10, cv2.LINE_AA) #빨간색 동그라미 그리기
                
                #각 특징에 내가 그린 이미지 그리기
                #image[ right_eye[1]-50:right_eye[1]+50, right_eye[0]-50:right_eye[0]+50 ]=image_right_eye
                #image[ left_eye[1]-50:left_eye[1]+50, left_eye[0]-50:left_eye[0]+50 ]=image_left_eye
                #image[ nose_tip[1]-50:nose_tip[1]+50, nose_tip[0]-150:nose_tip[0]+150 ]=image_nose
                
                #오버레이 함수 호출하기
                overlay(image, *right_eye, 50,50,image_right_eye) #오른쪽 눈
                overlay(image, *left_eye, 50,50,image_left_eye) #왼쪽 눈
                overlay(image, *nose_tip, 150,50,image_nose) #귀 눈
                
        #동영상 보여주는 부분
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5)) #처리된 이미지를 화면에 표시
        
        #q누르면 while문 종료
        if cv2.waitKey(1) == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

label_id: 0
score: 0.9720634
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.33977893
    ymin: 0.27596474
    width: 0.16592523
    height: 0.29497838
  }
  relative_keypoints {
    x: 0.39748585
    y: 0.35070306
  }
  relative_keypoints {
    x: 0.47279006
    y: 0.3487131
  }
  relative_keypoints {
    x: 0.44714963
    y: 0.41272855
  }
  relative_keypoints {
    x: 0.44401816
    y: 0.4775529
  }
  relative_keypoints {
    x: 0.33800027
    y: 0.39432424
  }
  relative_keypoints {
    x: 0.49792442
    y: 0.3847406
  }
}

label_id: 0
score: 0.96794623
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.331663
    ymin: 0.27528393
    width: 0.16455492
    height: 0.2925421
  }
  relative_keypoints {
    x: 0.38922748
    y: 0.3448243
  }
  relative_keypoints {
    x: 0.4635287
    y: 0.33934838
  }
  relative_keypoints {
    x: 0.4386563
    y: 0.39772916
  }
  relative_keypoints {
    x: 0.4356684
    y: 0.4

label_id: 0
score: 0.96779424
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.20344609
    ymin: 0.2489456
    width: 0.1775882
    height: 0.31571245
  }
  relative_keypoints {
    x: 0.26904845
    y: 0.3408128
  }
  relative_keypoints {
    x: 0.34652948
    y: 0.31829733
  }
  relative_keypoints {
    x: 0.33160505
    y: 0.39522833
  }
  relative_keypoints {
    x: 0.33139274
    y: 0.46483773
  }
  relative_keypoints {
    x: 0.2014082
    y: 0.399436
  }
  relative_keypoints {
    x: 0.3686855
    y: 0.3450786
  }
}

label_id: 0
score: 0.959568
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.20600326
    ymin: 0.24915403
    width: 0.17289792
    height: 0.30737394
  }
  relative_keypoints {
    x: 0.27365485
    y: 0.33535898
  }
  relative_keypoints {
    x: 0.34757137
    y: 0.3175363
  }
  relative_keypoints {
    x: 0.33333883
    y: 0.38960356
  }
  relative_keypoints {
    x: 0.33129203
    y: 0.4

label_id: 0
score: 0.94001424
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.33928624
    ymin: 0.2640229
    width: 0.1913068
    height: 0.3401013
  }
  relative_keypoints {
    x: 0.39441156
    y: 0.3820086
  }
  relative_keypoints {
    x: 0.4736319
    y: 0.3748538
  }
  relative_keypoints {
    x: 0.43799403
    y: 0.47280204
  }
  relative_keypoints {
    x: 0.44119766
    y: 0.52363276
  }
  relative_keypoints {
    x: 0.35086834
    y: 0.3892976
  }
  relative_keypoints {
    x: 0.52006584
    y: 0.36880177
  }
}

label_id: 0
score: 0.9314986
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.33952823
    ymin: 0.2650702
    width: 0.19133046
    height: 0.34014362
  }
  relative_keypoints {
    x: 0.396326
    y: 0.37985975
  }
  relative_keypoints {
    x: 0.4761923
    y: 0.37988305
  }
  relative_keypoints {
    x: 0.4398186
    y: 0.4757685
  }
  relative_keypoints {
    x: 0.4409473
    y: 0.52672

label_id: 0
score: 0.942213
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.23859577
    ymin: 0.22616017
    width: 0.19150136
    height: 0.3404457
  }
  relative_keypoints {
    x: 0.30700454
    y: 0.3352545
  }
  relative_keypoints {
    x: 0.38627592
    y: 0.3351487
  }
  relative_keypoints {
    x: 0.3550812
    y: 0.42526054
  }
  relative_keypoints {
    x: 0.34980717
    y: 0.48022348
  }
  relative_keypoints {
    x: 0.24429213
    y: 0.33724236
  }
  relative_keypoints {
    x: 0.41726425
    y: 0.3388155
  }
}

label_id: 0
score: 0.6107348
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.73150235
    ymin: 0.25456607
    width: 0.18097228
    height: 0.32172853
  }
  relative_keypoints {
    x: 0.7659207
    y: 0.35359162
  }
  relative_keypoints {
    x: 0.8077956
    y: 0.352359
  }
  relative_keypoints {
    x: 0.75724715
    y: 0.44593853
  }
  relative_keypoints {
    x: 0.77462256
    y: 0.50

label_id: 0
score: 0.95687747
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.25164875
    ymin: 0.19873929
    width: 0.19007036
    height: 0.33790237
  }
  relative_keypoints {
    x: 0.32048354
    y: 0.3089615
  }
  relative_keypoints {
    x: 0.4002575
    y: 0.30171216
  }
  relative_keypoints {
    x: 0.3732659
    y: 0.39385676
  }
  relative_keypoints {
    x: 0.3688367
    y: 0.44938725
  }
  relative_keypoints {
    x: 0.2550543
    y: 0.31739205
  }
  relative_keypoints {
    x: 0.4292282
    y: 0.3032797
  }
}

label_id: 0
score: 0.93870527
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.2581018
    ymin: 0.2111569
    width: 0.19108352
    height: 0.33970338
  }
  relative_keypoints {
    x: 0.32266614
    y: 0.3189972
  }
  relative_keypoints {
    x: 0.405525
    y: 0.3188386
  }
  relative_keypoints {
    x: 0.3728445
    y: 0.40632808
  }
  relative_keypoints {
    x: 0.36893904
    y: 0.4616

...

label_id: 0
score: 0.9586168
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.30503413
    ymin: 0.18522346
    width: 0.1596871
    height: 0.2838881
  }
  relative_keypoints {
    x: 0.36834013
    y: 0.2628895
  }
  relative_keypoints {
    x: 0.43290532
    y: 0.25095177
  }
  relative_keypoints {
    x: 0.41981295
    y: 0.31505728
  }
  relative_keypoints {
    x: 0.41851243
    y: 0.37904316
  }
  relative_keypoints {
    x: 0.3084266
    y: 0.3120787
  }
  relative_keypoints {
    x: 0.45225805
    y: 0.27740264
  }
}

label_id: 0
score: 0.5797789
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.75516397
    ymin: 0.28731298
    width: 0.1704492
    height: 0.30302095
  }
  relative_keypoints {
    x: 0.7997672
    y: 0.34052825
  }
  relative_keypoints {
    x: 0.864534
    y: 0.40184897
  }
  relative_keypoints {
    x: 0.7992241
    y: 0.44809437
  }
  relative_keypoints {
    x: 0.79019505
    y: 0.51

label_id: 0
score: 0.90281874
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.30383688
    ymin: 0.24850577
    width: 0.16960719
    height: 0.30152428
  }
  relative_keypoints {
    x: 0.37353694
    y: 0.34122062
  }
  relative_keypoints {
    x: 0.43943658
    y: 0.3407681
  }
  relative_keypoints {
    x: 0.42157906
    y: 0.41623306
  }
  relative_keypoints {
    x: 0.41525376
    y: 0.47091073
  }
  relative_keypoints {
    x: 0.3087655
    y: 0.35666102
  }
  relative_keypoints {
    x: 0.45590132
    y: 0.35107464
  }
}

label_id: 0
score: 0.90379757
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.30142117
    ymin: 0.24107581
    width: 0.17380852
    height: 0.30899346
  }
  relative_keypoints {
    x: 0.37407687
    y: 0.33381224
  }
  relative_keypoints {
    x: 0.44050586
    y: 0.33098185
  }
  relative_keypoints {
    x: 0.42489588
    y: 0.40692985
  }
  relative_keypoints {
    x: 0.4191905
  

label_id: 0
score: 0.9540024
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.31181526
    ymin: 0.19326699
    width: 0.16024011
    height: 0.28487164
  }
  relative_keypoints {
    x: 0.3691393
    y: 0.27487034
  }
  relative_keypoints {
    x: 0.4343579
    y: 0.26271164
  }
  relative_keypoints {
    x: 0.41622615
    y: 0.33129936
  }
  relative_keypoints {
    x: 0.4161488
    y: 0.39264983
  }
  relative_keypoints {
    x: 0.31698176
    y: 0.3155552
  }
  relative_keypoints {
    x: 0.46094647
    y: 0.2843358
  }
}

label_id: 0
score: 0.5384596
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.7488428
    ymin: 0.2913925
    width: 0.17180735
    height: 0.30543476
  }
  relative_keypoints {
    x: 0.78191644
    y: 0.39273965
  }
  relative_keypoints {
    x: 0.8404439
    y: 0.40079397
  }
  relative_keypoints {
    x: 0.7918193
    y: 0.48710352
  }
  relative_keypoints {
    x: 0.80034167
    y: 0.5

label_id: 0
score: 0.9254653
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.3604436
    ymin: 0.18539584
    width: 0.14867848
    height: 0.26431668
  }
  relative_keypoints {
    x: 0.41368616
    y: 0.25988823
  }
  relative_keypoints {
    x: 0.47548732
    y: 0.22881377
  }
  relative_keypoints {
    x: 0.4658908
    y: 0.28525865
  }
  relative_keypoints {
    x: 0.46925613
    y: 0.3523264
  }
  relative_keypoints {
    x: 0.36093867
    y: 0.33705384
  }
  relative_keypoints {
    x: 0.49752817
    y: 0.26733613
  }
}

label_id: 0
score: 0.53019124
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.74310106
    ymin: 0.28953576
    width: 0.21219677
    height: 0.3772375
  }
  relative_keypoints {
    x: 0.7932153
    y: 0.38976216
  }
  relative_keypoints {
    x: 0.86528015
    y: 0.44271928
  }
  relative_keypoints {
    x: 0.78959095
    y: 0.50094867
  }
  relative_keypoints {
    x: 0.7882703
    y:

label_id: 0
score: 0.946085
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.35340238
    ymin: 0.19233763
    width: 0.16044599
    height: 0.2852375
  }
  relative_keypoints {
    x: 0.41292953
    y: 0.2752707
  }
  relative_keypoints {
    x: 0.47955853
    y: 0.26390243
  }
  relative_keypoints {
    x: 0.4619916
    y: 0.3311491
  }
  relative_keypoints {
    x: 0.4615725
    y: 0.38991964
  }
  relative_keypoints {
    x: 0.3576086
    y: 0.31765276
  }
  relative_keypoints {
    x: 0.50246286
    y: 0.28629917
  }
}

label_id: 0
score: 0.9524639
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.3481736
    ymin: 0.19035423
    width: 0.15823528
    height: 0.28130692
  }
  relative_keypoints {
    x: 0.40511692
    y: 0.26664984
  }
  relative_keypoints {
    x: 0.47353736
    y: 0.26191962
  }
  relative_keypoints {
    x: 0.45113954
    y: 0.3232342
  }
  relative_keypoints {
    x: 0.4488573
    y: 0.38

In [3]:
import cv2
image_right_eye = cv2.imread('right_eye_trans.png', cv2.IMREAD_UNCHANGED)
image_right_eye.shape
#투명도를 넣어서 4채널이 됨. 채널값 확인해주기

(100, 100, 4)

label_id: 0
score: 0.9534881
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.36560073
    ymin: 0.3078007
    width: 0.3340889
    height: 0.44543034
  }
  relative_keypoints {
    x: 0.47420868
    y: 0.4237678
  }
  relative_keypoints {
    x: 0.60904115
    y: 0.44579703
  }
  relative_keypoints {
    x: 0.53712577
    y: 0.55061966
  }
  relative_keypoints {
    x: 0.52758676
    y: 0.6343054
  }
  relative_keypoints {
    x: 0.39066437
    y: 0.44297963
  }
  relative_keypoints {
    x: 0.67284197
    y: 0.48859996
  }
}

label_id: 0
score: 0.9564464
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.3650697
    ymin: 0.30908436
    width: 0.3353442
    height: 0.44710052
  }
  relative_keypoints {
    x: 0.47417316
    y: 0.4243837
  }
  relative_keypoints {
    x: 0.61005384
    y: 0.44506449
  }
  relative_keypoints {
    x: 0.53875715
    y: 0.54912233
  }
  relative_keypoints {
    x: 0.52942955
    y: 0

label_id: 0
score: 0.944891
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.3588951
    ymin: 0.30440056
    width: 0.33942184
    height: 0.4525382
  }
  relative_keypoints {
    x: 0.47098988
    y: 0.42440057
  }
  relative_keypoints {
    x: 0.6050628
    y: 0.4436493
  }
  relative_keypoints {
    x: 0.53452003
    y: 0.552831
  }
  relative_keypoints {
    x: 0.525683
    y: 0.6373143
  }
  relative_keypoints {
    x: 0.38791093
    y: 0.4448654
  }
  relative_keypoints {
    x: 0.6684993
    y: 0.4850285
  }
}

label_id: 0
score: 0.95386225
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.36030546
    ymin: 0.3110649
    width: 0.33493528
    height: 0.44655854
  }
  relative_keypoints {
    x: 0.46937284
    y: 0.42875093
  }
  relative_keypoints {
    x: 0.6048552
    y: 0.44632173
  }
  relative_keypoints {
    x: 0.53503114
    y: 0.5528752
  }
  relative_keypoints {
    x: 0.52659506
    y: 0.6375031

label_id: 0
score: 0.93278986
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.35352218
    ymin: 0.3232227
    width: 0.34670782
    height: 0.4622478
  }
  relative_keypoints {
    x: 0.45150316
    y: 0.45374507
  }
  relative_keypoints {
    x: 0.5887148
    y: 0.478014
  }
  relative_keypoints {
    x: 0.50459933
    y: 0.5970014
  }
  relative_keypoints {
    x: 0.50097036
    y: 0.676861
  }
  relative_keypoints {
    x: 0.38532257
    y: 0.45957798
  }
  relative_keypoints {
    x: 0.67246485
    y: 0.505975
  }
}

label_id: 0
score: 0.9338585
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.35418808
    ymin: 0.3224365
    width: 0.34660995
    height: 0.46211064
  }
  relative_keypoints {
    x: 0.45015907
    y: 0.45453244
  }
  relative_keypoints {
    x: 0.5869164
    y: 0.47876495
  }
  relative_keypoints {
    x: 0.50125504
    y: 0.59846824
  }
  relative_keypoints {
    x: 0.49840537
    y: 0.677

label_id: 0
score: 0.9373952
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.34390008
    ymin: 0.31100363
    width: 0.36126506
    height: 0.48164105
  }
  relative_keypoints {
    x: 0.4481927
    y: 0.44922507
  }
  relative_keypoints {
    x: 0.58852834
    y: 0.47845024
  }
  relative_keypoints {
    x: 0.5014078
    y: 0.6045763
  }
  relative_keypoints {
    x: 0.4960445
    y: 0.68562514
  }
  relative_keypoints {
    x: 0.37925273
    y: 0.44754308
  }
  relative_keypoints {
    x: 0.6722826
    y: 0.5033298
  }
}

label_id: 0
score: 0.9328444
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.34499827
    ymin: 0.31641203
    width: 0.35500416
    height: 0.47330034
  }
  relative_keypoints {
    x: 0.45031828
    y: 0.4509911
  }
  relative_keypoints {
    x: 0.5883503
    y: 0.4803325
  }
  relative_keypoints {
    x: 0.5042922
    y: 0.6028246
  }
  relative_keypoints {
    x: 0.4979522
    y: 0.6827

label_id: 0
score: 0.94342357
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.36018598
    ymin: 0.28523773
    width: 0.41705805
    height: 0.5560012
  }
  relative_keypoints {
    x: 0.4786661
    y: 0.47632718
  }
  relative_keypoints {
    x: 0.6495861
    y: 0.48254287
  }
  relative_keypoints {
    x: 0.5591
    y: 0.6492889
  }
  relative_keypoints {
    x: 0.5593798
    y: 0.72838086
  }
  relative_keypoints {
    x: 0.38956726
    y: 0.46949416
  }
  relative_keypoints {
    x: 0.7435404
    y: 0.47894257
  }
}

label_id: 0
score: 0.93328404
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.35611153
    ymin: 0.27717733
    width: 0.434304
    height: 0.5789841
  }
  relative_keypoints {
    x: 0.48365855
    y: 0.4724843
  }
  relative_keypoints {
    x: 0.65765786
    y: 0.4771403
  }
  relative_keypoints {
    x: 0.56830907
    y: 0.649707
  }
  relative_keypoints {
    x: 0.5688632
    y: 0.7351102


...

label_id: 0
score: 0.89017844
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.4877831
    ymin: 0.2657364
    width: 0.34187374
    height: 0.45580024
  }
  relative_keypoints {
    x: 0.5843429
    y: 0.38562304
  }
  relative_keypoints {
    x: 0.72469985
    y: 0.43021363
  }
  relative_keypoints {
    x: 0.6288529
    y: 0.53535384
  }
  relative_keypoints {
    x: 0.617918
    y: 0.6131596
  }
  relative_keypoints {
    x: 0.5148955
    y: 0.37813032
  }
  relative_keypoints {
    x: 0.80586606
    y: 0.46960175
  }
}

label_id: 0
score: 0.905577
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.48688287
    ymin: 0.26698655
    width: 0.34193337
    height: 0.4558804
  }
  relative_keypoints {
    x: 0.5841178
    y: 0.38750798
  }
  relative_keypoints {
    x: 0.72468793
    y: 0.43025196
  }
  relative_keypoints {
    x: 0.63056636
    y: 0.5357218
  }
  relative_keypoints {
    x: 0.6196265
    y: 0.6135

label_id: 0
score: 0.96778363
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.48346803
    ymin: 0.23163833
    width: 0.36444232
    height: 0.4858848
  }
  relative_keypoints {
    x: 0.5859661
    y: 0.36823684
  }
  relative_keypoints {
    x: 0.733727
    y: 0.40766376
  }
  relative_keypoints {
    x: 0.6345657
    y: 0.52752715
  }
  relative_keypoints {
    x: 0.6247196
    y: 0.60856456
  }
  relative_keypoints {
    x: 0.5133667
    y: 0.35855138
  }
  relative_keypoints {
    x: 0.81866646
    y: 0.4418707
  }
}

label_id: 0
score: 0.97040814
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.48204145
    ymin: 0.23141874
    width: 0.3669794
    height: 0.4892661
  }
  relative_keypoints {
    x: 0.58535886
    y: 0.36845326
  }
  relative_keypoints {
    x: 0.7341004
    y: 0.4071557
  }
  relative_keypoints {
    x: 0.6343916
    y: 0.52801156
  }
  relative_keypoints {
    x: 0.6242037
    y: 0.6104

label_id: 0
score: 0.9658545
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.47526348
    ymin: 0.22124483
    width: 0.3773377
    height: 0.5030653
  }
  relative_keypoints {
    x: 0.59643066
    y: 0.3641323
  }
  relative_keypoints {
    x: 0.74467945
    y: 0.40868258
  }
  relative_keypoints {
    x: 0.6518888
    y: 0.5345141
  }
  relative_keypoints {
    x: 0.6363581
    y: 0.6145002
  }
  relative_keypoints {
    x: 0.508564
    y: 0.34646362
  }
  relative_keypoints {
    x: 0.8153214
    y: 0.4390136
  }
}

label_id: 0
score: 0.96218944
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.47388938
    ymin: 0.21925227
    width: 0.3786148
    height: 0.50476795
  }
  relative_keypoints {
    x: 0.5976836
    y: 0.3680901
  }
  relative_keypoints {
    x: 0.7449325
    y: 0.41136235
  }
  relative_keypoints {
    x: 0.6546944
    y: 0.5420348
  }
  relative_keypoints {
    x: 0.63942266
    y: 0.61894774

label_id: 0
score: 0.9170679
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.44214994
    ymin: 0.20884778
    width: 0.3778826
    height: 0.5037984
  }
  relative_keypoints {
    x: 0.5656923
    y: 0.35188538
  }
  relative_keypoints {
    x: 0.71363777
    y: 0.3921227
  }
  relative_keypoints {
    x: 0.6244103
    y: 0.518545
  }
  relative_keypoints {
    x: 0.610112
    y: 0.60020643
  }
  relative_keypoints {
    x: 0.47547224
    y: 0.34156764
  }
  relative_keypoints {
    x: 0.7843219
    y: 0.42343754
  }
}

label_id: 0
score: 0.92382306
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.43966517
    ymin: 0.20767932
    width: 0.38066235
    height: 0.5075034
  }
  relative_keypoints {
    x: 0.5638127
    y: 0.3496127
  }
  relative_keypoints {
    x: 0.7134454
    y: 0.3896944
  }
  relative_keypoints {
    x: 0.6234465
    y: 0.5156174
  }
  relative_keypoints {
    x: 0.6090055
    y: 0.5997186
 

label_id: 0
score: 0.9557929
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.43506378
    ymin: 0.19778837
    width: 0.3975246
    height: 0.52996606
  }
  relative_keypoints {
    x: 0.56679904
    y: 0.350644
  }
  relative_keypoints {
    x: 0.7193648
    y: 0.39654094
  }
  relative_keypoints {
    x: 0.62478256
    y: 0.5300005
  }
  relative_keypoints {
    x: 0.6087157
    y: 0.6110566
  }
  relative_keypoints {
    x: 0.47378913
    y: 0.33530337
  }
  relative_keypoints {
    x: 0.79222876
    y: 0.4272005
  }
}

label_id: 0
score: 0.966257
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.43459693
    ymin: 0.20140056
    width: 0.39809397
    height: 0.53072524
  }
  relative_keypoints {
    x: 0.5641014
    y: 0.35404962
  }
  relative_keypoints {
    x: 0.71852434
    y: 0.39856154
  }
  relative_keypoints {
    x: 0.62223965
    y: 0.5327342
  }
  relative_keypoints {
    x: 0.6070875
    y: 0.6139